In [36]:
import pandas as pd
import glob
import os
import numpy as np

In [26]:
RESULT_DIR = "./results/results_vultr"

In [27]:
def get_latest_results_file(result_dir):    
    list_of_files = glob.glob(result_dir + '/*.csv') 
    latest_file = None
    if list_of_files:
        latest_file = max(list_of_files, key=os.path.getctime)
    return latest_file

In [28]:
results_file = get_latest_results_file(RESULT_DIR)
df = pd.read_csv(results_file)
display(df)

,dataset,optimizer,iteration,variation,runtime
0,FiftyWords,ssg-1000-1,0_2265492033579162998,210.526323,264.965515
1,FiftyWords,ssg-1000-1,1_1016076838762880639,210.542692,264.457735
2,FiftyWords,ssg-1000-1,2_729535312471433095,211.639740,263.875362
3,FiftyWords,ssg-1000-1,3_354463858690531473,216.683413,265.837008
4,FiftyWords,ssg-1000-1,4_956363461769994650,215.884868,265.260671
...,...,...,...,...,...
745,ChlorineConcentration,sgld-120,25_1909077642854329882,2.074789,239.621016
746,ChlorineConcentration,sgld-120,26_770383818710904589,2.531208,242.505795
747,ChlorineConcentration,sgld-120,27_155621029098090493,1.706476,239.733061
748,ChlorineConcentration,sgld-120,28_2194406957576155368,1.820859,235.883883


In [33]:
df.groupby(['dataset', 'optimizer']).agg(['mean', 'std'])

variation                 runtime  \
                                          mean         std        mean   
dataset               optimizer                                          
Adiac                 adam-1000-1   218.463104   98.058548  102.041313   
                      adam-1000-10  176.577020   80.002489  101.263556   
                      sgld-120       18.330223   10.924254   54.363168   
                      ssg-1000-1    114.768464    2.383617  102.427128   
                      ssg-1000-10   121.691856    7.487256  101.816744   
Beef                  adam-1000-1    52.536595   26.338912  452.553178   
                      adam-1000-10   98.449246  108.910657  452.231631   
                      sgld-120       -0.194802    0.105964   56.187078   
                      ssg-1000-1     18.954947    0.916854  428.917295   
                      ssg-1000-10    35.193129    8.448344  452.241868   
CBF                   adam-1000-1    23.419419    2.042729   61.406856   
                      adam-1000-10   29.911826    2.690424   61.763763   
                      sgld-120        2.018436    0.542033   33.981901   
                      ssg-1000-1     18.866242    0.284513   61.090787   
                      ssg-1000-10    21.955103    0.895854   61.144706   
ChlorineConcentration adam-1000-1    22.892403    2.586399  261.494552   
                      adam-1000-10   29.663537   33.331362  279.942609   
                      sgld-120        1.947024    0.562293  239.476087   
                      ssg-1000-1     32.730201   36.082698  262.596158   
                      ssg-1000-10    35.946530   43.534840  263.367830   
FiftyWords            adam-1000-1   346.276852  200.921966  265.657640   
                      adam-1000-10  525.583896  356.405120  265.076218   
                      sgld-120       19.747599   13.641258  146.299824   
                      ssg-1000-1    212.842595    3.313509  267.226628   
                      ssg-1000-10   229.629045   15.178742  263.378636   

                                               
                                          std  
dataset               optimizer                
Adiac                 adam-1000-1    0.666171  
                      adam-1000-10   0.623970  
                      sgld-120       0.426498  
                      ssg-1000-1     0.793085  
                      ssg-1000-10    0.780332  
Beef                  adam-1000-1    4.900081  
                      adam-1000-10   1.814511  
                      sgld-120       2.814370  
                      ssg-1000-1    76.194424  
                      ssg-1000-10    2.090803  
CBF                   adam-1000-1    0.419669  
                      adam-1000-10   0.492529  
                      sgld-120       0.237814  
                      ssg-1000-1     0.408084  
                      ssg-1000-10    0.486428  
ChlorineConcentration adam-1000-1    1.868626  
                      adam-1000-10  91.865838  
                      sgld-120       2.072343  
                      ssg-1000-1     1.466205  
                      ssg-1000-10    1.696079  
FiftyWords            adam-1000-1    1.280234  
                      adam-1000-10   3.049561  
                      sgld-120       1.170676  
                      ssg-1000-1    15.006884  
                      ssg-1000-10    1.651765

In [60]:
REFERENCE_FILE = "./5-sets_reference-results.csv"
df_r = pd.read_csv(REFERENCE_FILE)
df_r.groupby(['dataset']).mean()

,mean,std
dataset,,
Adiac,0.588,0.0140
Beef,23.830,3.4000
CBF,19.300,0.5940
ChlorineConcentration,15.396,0.5420
FiftyWords,21.382,1.7676


In [48]:
df['variation_sign'] = np.sign(df.variation)
df.groupby(['dataset', 'optimizer']).variation_sign.value_counts().unstack()

variation_sign                      -1.0   1.0
dataset               optimizer               
Adiac                 adam-1000-1    NaN  30.0
                      adam-1000-10   NaN  30.0
                      sgld-120       NaN  30.0
                      ssg-1000-1     NaN  30.0
                      ssg-1000-10    NaN  30.0
Beef                  adam-1000-1    NaN  30.0
                      adam-1000-10   NaN  30.0
                      sgld-120      30.0   NaN
                      ssg-1000-1     NaN  30.0
                      ssg-1000-10    NaN  30.0
CBF                   adam-1000-1    NaN  30.0
                      adam-1000-10   NaN  30.0
                      sgld-120       NaN  30.0
                      ssg-1000-1     NaN  30.0
                      ssg-1000-10    NaN  30.0
ChlorineConcentration adam-1000-1    NaN  30.0
                      adam-1000-10   NaN  30.0
                      sgld-120       NaN  30.0
                      ssg-1000-1     NaN  30.0
                      ssg-1000-10    NaN  30.0
FiftyWords            adam-1000-1    NaN  30.0
                      adam-1000-10   NaN  30.0
                      sgld-120       NaN  30.0
                      ssg-1000-1     NaN  30.0
                      ssg-1000-10    NaN  30.0